In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)#показывать все столбцы

# Первичный анализ выполненен в рамках файла /release 1.0/EDA.ipynb ввиду невозможности выполнения выполнения оценки данных из-за их высокой загрязненности. В данном файле был продолжен анализ, который был выполнен в EDA.ipynb

## Рефакторинг Release 2.0 в рамках Release 3.0

In [31]:
df=pd.read_csv("/home/alexandra/Documents/ds_bootcamp/HW/rent_real_estate/notebooks/data/data_new.csv")
df.head(3)

,ID,Parking,Apartment_renovation,Windows,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Lease_term,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony
0,271271157,подземная,Дизайнерский,NaN,4.0,9.0,200.0,500000.0,1,Срок аренды - Длительный,5.0,1.0,1.0,NaN,NaN
1,271634126,подземная,Дизайнерский,На улицу и двор,4.0,8.0,198.0,500000.0,1,Срок аренды - Длительный,2.0,1.0,0.0,3.0,NaN
2,271173086,подземная,Евроремонт,На улицу и двор,4.0,7.0,200.0,500000.0,0,NaN,1.0,1.0,0.0,3.0,NaN


# 1. Производим анализ процентного содержания пропущенных значений (NaN) в столбцах

In [32]:
for el in df.columns:
    if df[el].isna().sum()>0:
        print(f"'{el}' - cодержит - {df[el].isna().sum()} NaN. Процент от общего числа от общего числа значений - {(100*df[el].isna().sum()/len(df[el])):.2f} %")

'Parking' - cодержит - 11174 NaN. Процент от общего числа от общего числа значений - 56.61 %
'Apartment_renovation' - cодержит - 2463 NaN. Процент от общего числа от общего числа значений - 12.48 %
'Windows' - cодержит - 5150 NaN. Процент от общего числа от общего числа значений - 26.09 %
'Quantity_of_rooms' - cодержит - 535 NaN. Процент от общего числа от общего числа значений - 2.71 %
'Distance_min' - cодержит - 884 NaN. Процент от общего числа от общего числа значений - 4.48 %
'Lease_term' - cодержит - 2367 NaN. Процент от общего числа от общего числа значений - 11.99 %
'Elevator_total' - cодержит - 4192 NaN. Процент от общего числа от общего числа значений - 21.24 %
'With_children' - cодержит - 4915 NaN. Процент от общего числа от общего числа значений - 24.90 %
'With_pets' - cодержит - 4915 NaN. Процент от общего числа от общего числа значений - 24.90 %
'WashingRoom_total' - cодержит - 2041 NaN. Процент от общего числа от общего числа значений - 10.34 %
'Loggia_and_Balcony' - cоде

# 2. 'Parking'.
## В данном столбце мы наблюдаем, что более 56.61% значений являются неопределенными. В связи с этим можно сделать вывод о нецелесообразности заполнения пропусков, так как это может исказить истинные данные и повлиять на надежность анализа.

In [33]:
df=df.drop('Parking',axis=1)
df.head(3)

,ID,Apartment_renovation,Windows,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Lease_term,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony
0,271271157,Дизайнерский,NaN,4.0,9.0,200.0,500000.0,1,Срок аренды - Длительный,5.0,1.0,1.0,NaN,NaN
1,271634126,Дизайнерский,На улицу и двор,4.0,8.0,198.0,500000.0,1,Срок аренды - Длительный,2.0,1.0,0.0,3.0,NaN
2,271173086,Евроремонт,На улицу и двор,4.0,7.0,200.0,500000.0,0,NaN,1.0,1.0,0.0,3.0,NaN


# 3. Apartment_renovation. 
## Для начала было решено добавить колонку стоимость квадратного метра.

In [34]:
df['Price_per_square_meter'] = (df['Cost_per_month'] / df['Area_sm']).round(2)
df.head(3)


,ID,Apartment_renovation,Windows,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Lease_term,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony,Price_per_square_meter
0,271271157,Дизайнерский,NaN,4.0,9.0,200.0,500000.0,1,Срок аренды - Длительный,5.0,1.0,1.0,NaN,NaN,2500.00
1,271634126,Дизайнерский,На улицу и двор,4.0,8.0,198.0,500000.0,1,Срок аренды - Длительный,2.0,1.0,0.0,3.0,NaN,2525.25
2,271173086,Евроремонт,На улицу и двор,4.0,7.0,200.0,500000.0,0,NaN,1.0,1.0,0.0,3.0,NaN,2500.00


# 4. Заполнение значений в столбцах Elevator_total и Distance_min медианой. 
## Это помогает сохранить качество данных. Медиана меньше подвержена влиянию выбросов, поэтому она лучше отражает типичные значения. Это важно, чтобы избежать искажений, которые могут возникнуть при использовании средних значений. Заполняя пропуски медианой, мы получаем более точное представление о данных и улучшаем их анализ.

In [35]:
df['Elevator_total'].fillna(df['Elevator_total'].median(), inplace=True)
df['Distance_min'].fillna(df['Distance_min'].median(), inplace=True)
print("Обновленный DataFrame:")
df.head(3)

Обновленный DataFrame:


/tmp/ipykernel_75420/352654931.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Elevator_total'].fillna(df['Elevator_total'].median(), inplace=True)
/tmp/ipykernel_75420/352654931.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

,ID,Apartment_renovation,Windows,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Lease_term,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony,Price_per_square_meter
0,271271157,Дизайнерский,NaN,4.0,9.0,200.0,500000.0,1,Срок аренды - Длительный,5.0,1.0,1.0,NaN,NaN,2500.00
1,271634126,Дизайнерский,На улицу и двор,4.0,8.0,198.0,500000.0,1,Срок аренды - Длительный,2.0,1.0,0.0,3.0,NaN,2525.25
2,271173086,Евроремонт,На улицу и двор,4.0,7.0,200.0,500000.0,0,NaN,1.0,1.0,0.0,3.0,NaN,2500.00


# 5.Заполняем "нулями" отсутствующие значения для квартир с "детьми" и "животными". 
## Отсутствующие значения в столбцах, отражающих наличие детей и животных, заполняются нулями. Это основано на предположении, что если бы информация о наличии детей или домашних животных была актуальной для квартиры, она была бы явно указана. Таким образом, значение "0" представляет собой отсутствие этих характеристик, что помогает упростить анализ и интерпретацию данных, не вводя при этом дополнительных искажений.

In [36]:
df['With_children'].fillna(0, inplace=True)
df['With_pets'].fillna(0, inplace=True)
print("Обновленный DataFrame:")
df.head(3)

Обновленный DataFrame:


/tmp/ipykernel_75420/3634273341.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['With_children'].fillna(0, inplace=True)
/tmp/ipykernel_75420/3634273341.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

,ID,Apartment_renovation,Windows,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Lease_term,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony,Price_per_square_meter
0,271271157,Дизайнерский,NaN,4.0,9.0,200.0,500000.0,1,Срок аренды - Длительный,5.0,1.0,1.0,NaN,NaN,2500.00
1,271634126,Дизайнерский,На улицу и двор,4.0,8.0,198.0,500000.0,1,Срок аренды - Длительный,2.0,1.0,0.0,3.0,NaN,2525.25
2,271173086,Евроремонт,На улицу и двор,4.0,7.0,200.0,500000.0,0,NaN,1.0,1.0,0.0,3.0,NaN,2500.00


# 4. Мы применили функцию fill_missing_values_by_median для заполнения пропусков в различных столбцах DataFrame, используя медианные значения из других столбцов.

### Количество комнат (Quantity_of_rooms): Заполнили пропуски на основе медианной площади квартиры (Area_sm). Это позволяет учесть, что размер квартиры часто коррелирует с количеством комнат.
### Количество санузлов (WashingRoom_total): Заполнили значения на основе количества комнат, предполагая, что большее количество комнат обычно связано с большим числом санузлов.
### Количество балконов (Loggia_and_Balcony): Аналогично, пропуски заполнялись на основе количества комнат, что помогает сохранить логику распределения данных.
### Состояние ремонта (Apartment_renovation): Здесь мы заполнили пропуски на основе цены за квадратный метр (Price_per_square_meter), поскольку стоимость может отражать состояние квартиры.
### Срок аренды (Lease_term): Пропуски были заполнены также на основе цены за квадратный метр, чтобы учесть зависимость между арендной стоимостью и сроком аренды.
### Окна (Windows): Указание произвольных значений могло бы вводить в заблуждение, так как реальная информация о состоянии окон неизвестна. Использование "Нет информации" позволяет точно отразить отсутствие данных, сохраняя целостность анализа и позволяя избежать ложных выводов.

In [37]:
def fill_missing_values_by_median(df, group_col, value_col):
    """
    Функция для заполнения пропусков в столбце group_col на основе медианы столбца value_col.
    """
    table_quantity = df.groupby(group_col).agg(Med_value=(value_col, 'median')).reset_index()
    df_filtered = df[df[group_col].isna()]

    for idx, row in df_filtered.iterrows():
        add_value = row[value_col]
        table_quantity['delta'] = abs(add_value - table_quantity['Med_value'])
        closest_group_value = table_quantity.loc[table_quantity['delta'].idxmin(), group_col]
        df.at[idx, group_col] = closest_group_value

    return df



df = fill_missing_values_by_median(df, 'Quantity_of_rooms', 'Area_sm')
df = fill_missing_values_by_median(df, 'WashingRoom_total', 'Quantity_of_rooms')
df = fill_missing_values_by_median(df, 'Loggia_and_Balcony', 'Quantity_of_rooms')
df = fill_missing_values_by_median(df, 'Apartment_renovation', 'Price_per_square_meter') 
df = fill_missing_values_by_median(df, 'Lease_term', 'Price_per_square_meter') 
df['Windows'].fillna("Нет информации", inplace=True)

print("Обновленный DataFrame:")
df.head(3)

Обновленный DataFrame:


/tmp/ipykernel_75420/3660935688.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Windows'].fillna("Нет информации", inplace=True)


,ID,Apartment_renovation,Windows,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Lease_term,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony,Price_per_square_meter
0,271271157,Дизайнерский,Нет информации,4.0,9.0,200.0,500000.0,1,Срок аренды - Длительный,5.0,1.0,1.0,3.0,4.0,2500.00
1,271634126,Дизайнерский,На улицу и двор,4.0,8.0,198.0,500000.0,1,Срок аренды - Длительный,2.0,1.0,0.0,3.0,4.0,2525.25
2,271173086,Евроремонт,На улицу и двор,4.0,7.0,200.0,500000.0,0,Срок аренды - На несколько месяцев,1.0,1.0,0.0,3.0,4.0,2500.00


Теперь мы можем снова запустить проверку, чтобы убедиться, что в нашем DataFrame действительно не осталось столбцов с пропущенными значениями. Это позволит подтвердить успешность проведенной обработки данных и подготовленность их для дальнейшего анализа.

In [38]:
for el in df.columns:
    if df[el].isna().sum()>0:
        print(f"'{el}' - cодержит - {df[el].isna().sum()} NaN. Процент от общего числа от общего числа значений - {(100*df[el].isna().sum()/len(df[el])):.2f} %")

Как видно, в нашем DataFrame больше нет столбцов с пропущенными значениями (NaN). Это достижение свидетельствует о проведенной тщательной обработке данных, что позволяет нам уверенно продолжать анализ и извлекать ценные инсайты без риска искажений, связанных с отсутствующими значениями.

In [39]:
# df.to_csv("data.csv", index=False, encoding='utf-8')

При извлечении файла data.csv мы тщательно проверили его на соответствие заданным критериям. Все названия колонок представлены на английском языке и оформлены в формате с использованием нижнего подчеркивания, что обеспечивает единообразие и удобство работы с данными. Кроме того, в каждой колонке отсутствуют пропущенные значения (NaN, None и т.д.), что подтверждает полную целостность и готовность данных к дальнейшему анализу. Таким образом, файл успешно отвечает всем предъявленным требованиям, позволяя уверенно использовать его в последующих этапах работы.

# Работа над seaborn_data.csv

## 1. Оценка корректности данных в нашем DataFrame 
### На данном этапе мы будем:

Проверять соответствие данных ожидаемым типам (числовые значения и категории).
Анализировать названия столбцов на предмет корректности и согласованности

In [40]:
df.head()

,ID,Apartment_renovation,Windows,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Lease_term,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony,Price_per_square_meter
0,271271157,Дизайнерский,Нет информации,4.0,9.0,200.0,500000.0,1,Срок аренды - Длительный,5.0,1.0,1.0,3.0,4.0,2500.00
1,271634126,Дизайнерский,На улицу и двор,4.0,8.0,198.0,500000.0,1,Срок аренды - Длительный,2.0,1.0,0.0,3.0,4.0,2525.25
2,271173086,Евроремонт,На улицу и двор,4.0,7.0,200.0,500000.0,0,Срок аренды - На несколько месяцев,1.0,1.0,0.0,3.0,4.0,2500.00
3,272197456,Евроремонт,На улицу и двор,4.0,3.0,170.0,400000.0,0,Срок аренды - На несколько месяцев,1.0,0.0,1.0,3.0,4.0,2352.94
4,273614615,Евроремонт,На улицу и двор,2.0,7.0,58.0,225000.0,1,Срок аренды - Длительный,2.0,0.0,0.0,2.0,1.0,3879.31


In [41]:
df.dtypes

ID                               int64
Apartment_renovation            object
Windows                         object
Quantity_of_rooms              float64
Distance_min                   float64
Area_sm                        float64
Cost_per_month                 float64
Is_included_utility_charges      int64
Lease_term                      object
Elevator_total                 float64
With_children                  float64
With_pets                      float64
WashingRoom_total              float64
Loggia_and_Balcony             float64
Price_per_square_meter         float64
dtype: object

# 2. Вывод о несоответствии типов данных

## На основании проведённой оценки мы пришли к выводу, что следующие колонки не соответствуют требуемому типу данных:

### Apartment_renovation
### Windows
### Lease_term
В настоящее время эти столбцы имеют тип object, что не соответствует ожидаемым числовым значениям. Необходимо преобразовать их в числовой тип данных для дальнейшего анализа.

# 2.1 Формирование списков для преобразования типов данных

## В ходе анализа были сформированы два списка, содержащие столбцы с типом Object:

### list_2: Столбцы, которые можно преобразовать в бинарные значения (0 или 1).
### list_3: Столбцы, для которых будут созданы новые категориальные столбцы.
Данное разделение поможет в дальнейшей обработке и анализе данных, обеспечивая их корректное представление в числовом формате.

In [42]:
list2=[]
list3=[]
for col in df.columns:
    if (df.dtypes[col]!='int64') and (df.dtypes[col]!='float64'):
        if df[col].nunique()==2:
            list2.append(col) #список значений, которые можно преобразовать в один столбец 0 или 1
        else:
            list3.append(col) #список значений, для которых нужен другой способ
print(list2)
print(list3)

['Lease_term']
['Apartment_renovation', 'Windows']


In [43]:

def code_myohe(data, feature):
    for i in data[feature].unique():
        data[feature + '=' + i] = (data[feature] == i).astype(float)

for i in list3:
    code_myohe(df, i)

df

,ID,Apartment_renovation,Windows,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Lease_term,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony,Price_per_square_meter,Apartment_renovation=Дизайнерский,Apartment_renovation=Евроремонт,Apartment_renovation=Косметический,Apartment_renovation=Без ремонта,Windows=Нет информации,Windows=На улицу и двор,Windows=Во двор,Windows=На улицу
0,271271157,Дизайнерский,Нет информации,4.0,9.0,200.0,500000.0,1,Срок аренды - Длительный,5.0,1.0,1.0,3.0,4.0,2500.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,271634126,Дизайнерский,На улицу и двор,4.0,8.0,198.0,500000.0,1,Срок аренды - Длительный,2.0,1.0,0.0,3.0,4.0,2525.25,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,271173086,Евроремонт,На улицу и двор,4.0,7.0,200.0,500000.0,0,Срок аренды - На несколько месяцев,1.0,1.0,0.0,3.0,4.0,2500.00,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,272197456,Евроремонт,На улицу и двор,4.0,3.0,170.0,400000.0,0,Срок аренды - На несколько месяцев,1.0,0.0,1.0,3.0,4.0,2352.94,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,273614615,Евроремонт,На улицу и двор,2.0,7.0,58.0,225000.0,1,Срок аренды - Длительный,2.0,0.0,0.0,2.0,1.0,3879.31,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19732,215565511,Евроремонт,Нет информации,1.0,8.0,35.0,42000.0,1,Срок аренды - Длительный,2.0,0.0,0.0,1.0,1.0,1200.00,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
19733,274654844,Евроремонт,Во двор,1.0,7.0,38.7,45000.0,1,Срок аренды - Длительный,2.0,0.0,0.0,1.0,1.0,1162.79,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
19734,268679909,Дизайнерский,На улицу и двор,2.0,6.0,43.1,50000.0,1,Срок аренды - Длительный,2.0,1.0,0.0,1.0,1.0,1160.09,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19735,274807525,Евроремонт,Во двор,2.0,11.0,52.5,55000.0,1,Срок аренды - Длительный,3.0,0.0,0.0,2.0,1.0,1047.62,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


# 2.2 Обработка качественных данных с использованием кодирования
### Для обработки качественных данных мы применяем метод, который позволяет преобразовать категориальные значения в числовые. Это упрощает дальнейшую работу с данными и повышает их анализируемость.
### Используется следующий подход:
После выполнения этого кода в нашей таблице будут добавлены новые столбцы с преобразованными значениями, что поможет привести данные в порядок.

In [44]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in list2:
    le.fit(df[i])
    df[i+'_le'] = le.transform(df[i])

df.head(3)


,ID,Apartment_renovation,Windows,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Lease_term,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony,Price_per_square_meter,Apartment_renovation=Дизайнерский,Apartment_renovation=Евроремонт,Apartment_renovation=Косметический,Apartment_renovation=Без ремонта,Windows=Нет информации,Windows=На улицу и двор,Windows=Во двор,Windows=На улицу,Lease_term_le
0,271271157,Дизайнерский,Нет информации,4.0,9.0,200.0,500000.0,1,Срок аренды - Длительный,5.0,1.0,1.0,3.0,4.0,2500.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
1,271634126,Дизайнерский,На улицу и двор,4.0,8.0,198.0,500000.0,1,Срок аренды - Длительный,2.0,1.0,0.0,3.0,4.0,2525.25,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,271173086,Евроремонт,На улицу и двор,4.0,7.0,200.0,500000.0,0,Срок аренды - На несколько месяцев,1.0,1.0,0.0,3.0,4.0,2500.00,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1


# 2.3 Удаление ненужных столбцов
### После преобразования данных мы удаляем первоначальные столбцы, которые больше не нужны для анализа. Это позволяет упростить таблицу и улучшить читаемость данных.
### Таким образом, мы освобождаем пространство и оставляем только необходимые данные для дальнейшей работы.

In [45]:
df=df.drop(["Windows","Apartment_renovation","Lease_term"],axis=1)

In [46]:
df.head()

,ID,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony,Price_per_square_meter,Apartment_renovation=Дизайнерский,Apartment_renovation=Евроремонт,Apartment_renovation=Косметический,Apartment_renovation=Без ремонта,Windows=Нет информации,Windows=На улицу и двор,Windows=Во двор,Windows=На улицу,Lease_term_le
0,271271157,4.0,9.0,200.0,500000.0,1,5.0,1.0,1.0,3.0,4.0,2500.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
1,271634126,4.0,8.0,198.0,500000.0,1,2.0,1.0,0.0,3.0,4.0,2525.25,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,271173086,4.0,7.0,200.0,500000.0,0,1.0,1.0,0.0,3.0,4.0,2500.00,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
3,272197456,4.0,3.0,170.0,400000.0,0,1.0,0.0,1.0,3.0,4.0,2352.94,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
4,273614615,2.0,7.0,58.0,225000.0,1,2.0,0.0,0.0,2.0,1.0,3879.31,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0


# 2.4 Форматирование названий колонок
## На данном этапе мы приводим названия колонок в соответствие с условиями задачи. Это включает в себя стандартизацию имен столбцов для удобства работы с данными.

In [47]:
# Удаление лишних пробелов в названиях столбцов
df.columns = df.columns.str.strip()

# Переименование колонок
df = df.rename(columns={
    "Apartment_renovation=Дизайнерский": "is_Apartment_renovation_Designer",
    "Apartment_renovation=Евроремонт": "is_Apartment_renovation_Euro_remont",
    "Apartment_renovation=Косметический": "is_Apartment_renovation_Cosmetic",
    "Apartment_renovation=Без ремонта": "Is_No_renovation",
    "Windows=Нет информации": "is_Windows_No_information",
    "Windows=На улицу и двор": "is_Windows_Outside_and_Courtyard",
    "Windows=Во двор": "is_Windows_Courtyard",
    "Windows=На улицу": "is_Windows_Outside",
    "Lease_term_le": "Is_short_Lease_term"
})

df.head()  # Чтобы проверить результат

,ID,Quantity_of_rooms,Distance_min,Area_sm,Cost_per_month,Is_included_utility_charges,Elevator_total,With_children,With_pets,WashingRoom_total,Loggia_and_Balcony,Price_per_square_meter,is_Apartment_renovation_Designer,is_Apartment_renovation_Euro_remont,is_Apartment_renovation_Cosmetic,Is_No_renovation,is_Windows_No_information,is_Windows_Outside_and_Courtyard,is_Windows_Courtyard,is_Windows_Outside,Is_short_Lease_term
0,271271157,4.0,9.0,200.0,500000.0,1,5.0,1.0,1.0,3.0,4.0,2500.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
1,271634126,4.0,8.0,198.0,500000.0,1,2.0,1.0,0.0,3.0,4.0,2525.25,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,271173086,4.0,7.0,200.0,500000.0,0,1.0,1.0,0.0,3.0,4.0,2500.00,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
3,272197456,4.0,3.0,170.0,400000.0,0,1.0,0.0,1.0,3.0,4.0,2352.94,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
4,273614615,2.0,7.0,58.0,225000.0,1,2.0,0.0,0.0,2.0,1.0,3879.31,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0


# 2.5 Контрольная проверка типов данных

## На данном этапе мы выполняем контрольную проверку типов данных в нашем DataFrame. Это позволяет убедиться, что все столбцы имеют ожидаемые типы данных и что они соответствуют критериям задачи.

In [48]:
df.dtypes

ID                                       int64
Quantity_of_rooms                      float64
Distance_min                           float64
Area_sm                                float64
Cost_per_month                         float64
Is_included_utility_charges              int64
Elevator_total                         float64
With_children                          float64
With_pets                              float64
WashingRoom_total                      float64
Loggia_and_Balcony                     float64
Price_per_square_meter                 float64
is_Apartment_renovation_Designer       float64
is_Apartment_renovation_Euro_remont    float64
is_Apartment_renovation_Cosmetic       float64
Is_No_renovation                       float64
is_Windows_No_information              float64
is_Windows_Outside_and_Courtyard       float64
is_Windows_Courtyard                   float64
is_Windows_Outside                     float64
Is_short_Lease_term                      int64
dtype: object

### Как мы можем видеть: типы данных соответсвуют ожидаемым

# 2.6 Проверка на дубликаты

## На данном этапе мы выполняем проверку на наличие дублирующихся записей в нашем DataFrame. Это необходимо для обеспечения уникальности объявлений и корректности анализа данных.

In [49]:
duplicates = df.duplicated().sum()
print(f'Количество дубликатов: {duplicates}')

Количество дубликатов: 0


### Как мы можем видеть: дубликаты отсутствуют

# 2.7 Выгрузка данных в CSV файл

## На данном этапе мы сохраняем подготовленные данные в CSV файл. Это позволяет удобно хранить и передавать набор данных для дальнейшего анализа и использования.

In [50]:
df.to_csv("seaborn_data.csv", index=False, encoding='utf-8')

### В результате выполненной работы была сформирована таблица, полностью соответствующая условиям задачи. Мы провели необходимые преобразования данных, включая кодирование категориальных признаков и форматирование названий колонок, что позволило создать удобный для анализа и моделирования набор данных.